In [5]:
import os
import time
import torch
import pytorch_mask_rcnn as pmr
    
    
def main(args):
    device = torch.device("cuda" if torch.cuda.is_available() and args.use_cuda else "cpu")
    cuda = device.type == "cuda"
    if cuda: pmr.get_gpu_prop(show=True)
    print("\ndevice: {}".format(device))
    
    # ---------------------- prepare data loader ------------------------------- #
    
    dataset_test = pmr.datasets(args.dataset, args.data_dir, "val2017", train=True) # set train=True for eval
    sampler_test = torch.utils.data.SequentialSampler(dataset_test)

    batch_sampler_test = pmr.GroupedBatchSampler(
        sampler_test, dataset_test.aspect_ratios, args.batch_size)

    args.num_workers = min(os.cpu_count() // 2, 8, args.batch_size if args.batch_size > 1 else 0)
    data_loader_test = torch.utils.data.DataLoader(
        dataset_test, batch_sampler=batch_sampler_test, num_workers=args.num_workers,  
        collate_fn=pmr.collate_wrapper, pin_memory=cuda)

    d_test = pmr.DataPrefetcher(data_loader_test) if cuda else data_loader_test
    
    # -------------------------------------------------------------------------- #

    print(args)
    
    num_classes = len(d_test.dataset.classes)
    model = pmr.maskrcnn_resnet50(False, num_classes).to(device)
    
    checkpoint = torch.load(args.ckpt_path, map_location=device)
    model.load_state_dict(checkpoint["model"])
    print(checkpoint["eval_info"])
    del checkpoint
    torch.cuda.empty_cache()

    print("evaluating only...")
    B = time.time()
    eval_output, iter_eval = pmr.evaluate(model, d_test, device, args)
    B = time.time() - B
    print(eval_output)
    print("\ntotal time of this evaluation: {:.2f} s, speed: {:.2f} FPS".format(B, args.batch_size / iter_eval))
    
    
if __name__ == "__main__":
    import argparse
    parser = argparse.ArgumentParser()
    parser.add_argument("--dataset", default="coco") #
    parser.add_argument("--data-dir")
    parser.add_argument("--batch-size", type=int, default=2) #
    parser.add_argument("--iters", type=int, default=-1) #
    
    args = parser.parse_args([]) # for Jupyter
    args.use_cuda = True
    args.distributed = False
    
    #args.data_dir = "E:/PyTorch/data/coco2017"
    args.data_dir = "/data/nextcloud/dbc2017/files/jupyter/input/data/coco2017"
    #args.ckpt_path = "E:/PyTorch/scripts/ckpt/Remax_yolo_v5_small_coco-149.pth"
    args.ckpt_path = "/data/nextcloud/dbc2017/files/jupyter/input/ckpt/re_yolo_v5_small_coco-88.pth"
    args.results = os.path.join(os.path.dirname(args.ckpt_path), "results.pth")
    
    main(args)
    print("All over!")
    

cuda: True
available GPU(s): 1
0: {'name': 'GeForce RTX 2080 Ti', 'capability': [7, 5], 'total_momory': 10.76, 'sm_count': 68}

device: cuda
Nvidia DALI is utilized!
Namespace(batch_size=32, ckpt_path='/data/nextcloud/dbc2017/files/jupyter/input/ckpt/re_yolo_v5_small_coco-88.pth', data_dir='/data/nextcloud/dbc2017/files/jupyter/input/data/coco2017', dataset='coco', distributed=False, img_sizes=[640, 640], iters=-1, model_size='small', results='/data/nextcloud/dbc2017/files/jupyter/input/ckpt/results.pth', use_cuda=True)
IoU metric: bbox
 Average Precision  (AP) @[ IoU=0.50:0.95 | area=   all | maxDets=100 ] = 0.227
 Average Precision  (AP) @[ IoU=0.50      | area=   all | maxDets=100 ] = 0.360
 Average Precision  (AP) @[ IoU=0.75      | area=   all | maxDets=100 ] = 0.244
 Average Precision  (AP) @[ IoU=0.50:0.95 | area= small | maxDets=100 ] = 0.063
 Average Precision  (AP) @[ IoU=0.50:0.95 | area=medium | maxDets=100 ] = 0.246
 Average Precision  (AP) @[ IoU=0.50:0.95 | area= large |